In [13]:
import numpy as np

def linear_kernel(xi,xj):
    return(np.dot(xi,xj))


def poli_kernel(xi,xj,p):
    return (np.dot(xi,xj)+1)**p


def RBF_kernel(xi,xj,sigma):
    norm = np.linalg.norm(xi-xj)
    return np.exp(-norm**2/(2*sigma**2))



















